In [7]:
import os
import torch
import torch.nn as nn
import timm
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [ ]:
class CTRDataset(Dataset):
    def __init__(self, folder_path, transform=None, ctr_min=0.0, ctr_max=0.27):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)
        self.transform = transform
        self.ctr_min = ctr_min
        self.ctr_max = ctr_max

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        ctr, _, _ = filename.split("_", 2)
        ctr = float(ctr) / 100.0 
        ctr = (ctr - self.ctr_min) / (self.ctr_max - self.ctr_min)
        img_path = os.path.join(self.folder_path, filename)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([ctr], dtype=torch.float32)

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


dataset = CTRDataset("filtered_images", transform)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:

model = timm.create_model("deit_small_patch16_224", pretrained=True)

for param in model.parameters():
    param.requires_grad = False

in_features = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 1)
)


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.head.parameters(), lr=1e-3)

# Training loop
best_val_loss = float('inf')
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for imgs, targets in train_loader:
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            all_preds.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())

    # Metrics
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


Epoch 1, Train Loss: 0.0230, Val Loss: 0.0180, MAE: 0.0906, R²: -0.4057
Epoch 2, Train Loss: 0.0172, Val Loss: 0.0147, MAE: 0.0896, R²: -0.1388
Epoch 3, Train Loss: 0.0100, Val Loss: 0.0132, MAE: 0.0783, R²: -0.0383
Epoch 4, Train Loss: 0.0092, Val Loss: 0.0113, MAE: 0.0760, R²: 0.1046
Epoch 5, Train Loss: 0.0087, Val Loss: 0.0116, MAE: 0.0715, R²: 0.0772
Epoch 6, Train Loss: 0.0089, Val Loss: 0.0127, MAE: 0.0792, R²: -0.0031
Epoch 7, Train Loss: 0.0070, Val Loss: 0.0123, MAE: 0.0726, R²: 0.0247
Epoch 8, Train Loss: 0.0065, Val Loss: 0.0132, MAE: 0.0735, R²: -0.0354
Epoch 9, Train Loss: 0.0072, Val Loss: 0.0118, MAE: 0.0790, R²: 0.0812
Epoch 10, Train Loss: 0.0082, Val Loss: 0.0129, MAE: 0.0752, R²: -0.0198


In [2]:
import os

# Define the path
path = r"C:\Users\benas\OneDrive\Desktop\University\Bachelor\solution\data\maxima\filtered_images"

# Supported image extensions
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}

# Count image files
image_count = sum(
    1 for file in os.listdir(path)
    if os.path.isfile(os.path.join(path, file)) and os.path.splitext(file)[1].lower() in image_extensions
)

print(f"Number of images: {image_count}")

Number of images: 675
